# Import Python libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from copy import deepcopy
import matplotlib.pyplot as plt
import matplotlib
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Load data

In [2]:
# load raw survey and building condition data
data = pd.read_csv('data/totalSurveyData.csv')
data.head()

,Timestamp,User ID,voteLog,Thermal Acceptance,Thermal Preference,Air Movement Preference,Demographics,Mode,Survey Type,StartDate,Condition,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,Thermal Sensation,Thermal Acceptabil,Thermal Preference.1,Space Preference_1,Reason for Rating,Air Freshness,Airflow Sensation,Airflow Preference,Air Humidity Sensati,Air Humidity Prefere,Indoor Air Quality,Indoor Air Quality.1,Indoor Air Quality_10_TEXT,Acoustics_1,Acoustics_2,Acoustics_3,Visual,Visual.1,BCA Test Site_AirReadings_Off Coil Temp,BCA Test Site_AirReadings_Outdoor Air RH,BCA Test Site_AirReadings_Outdoor Air Temp,BCA Test Site_AirReadings_Return Air CO2,BCA Test Site_AirReadings_Return Air RH,BCA Test Site_AirReadings_Return Air Temp,BCA Test Site_AirReadings_Supply Air CO2,BCA Test Site_AirReadings_Supply Air RH,BCA Test Site_AirReadings_Supply Air Temp,BCA Test Site_BTUReadings_BTU Cooling Load W,BCA Test Site_MainLinePowerMeter_Power W,BCA Test Site_PowerMeterReadings_AHU_Power W,BCA Test Site_PowerMeterReadings_Fan_Power W,BCA Test Site_VAVFeedback_VAV-01_Airflow Final,BCA Test Site_VAVFeedback_VAV-01_Temp,BCA Test Site_VAVFeedback_VAV-02_Airflow Final,BCA Test Site_VAVFeedback_VAV-02_Temp,BCA Test Site_VAVFeedback_VAV-03_Airflow Final,BCA Test Site_VAVFeedback_VAV-03_Temp,BCA Test Site_VAVFeedback_VAV-04_Airflow Final,BCA Test Site_VAVFeedback_VAV-04_Temp,UFL.AWAIR 33482 CO2,UFL.AWAIR 33482 HUMIDITY,UFL.AWAIR 33482 LUX,UFL.AWAIR 33482 PM10,UFL.AWAIR 33482 PM2.5,UFL.AWAIR 33482 SCORE,UFL.AWAIR 33482 SOUND,UFL.AWAIR 33482 TEMP,UFL.AWAIR 33482 VOC,UFL.AWAIR 33869 CO2,UFL.AWAIR 33869 HUMIDITY,UFL.AWAIR 33869 LUX,UFL.AWAIR 33869 PM10,UFL.AWAIR 33869 PM2.5,UFL.AWAIR 33869 SCORE,UFL.AWAIR 33869 SOUND,UFL.AWAIR 33869 TEMP,UFL.AWAIR 33869 VOC,UFL.Purple Air Outdoor 145432 Humidity,UFL.Purple Air Outdoor 145432 PM1,UFL.Purple Air Outdoor 145432 PM10,UFL.Purple Air Outdoor 145432 PM2.5,UFL.Purple Air Outdoor 145432 Pressure,UFL.Purple Air Outdoor 145432 Temperature,PPM55 Temperature,PPM55 Relative Humidity,PPM55 CO,PPM55 CO2,PPM55 TVOC,PPM55 Test Number,PPM55 Air Velocity,PPM54 Temperature,PPM54 Relative Humidity,PPM54 CO,PPM54 CO2,PPM54 TVOC,PPM54 Test Number,PPM54 Air Velocity,Testo400 Globe Temerature,Testo400 Temperature,Testo400 Air Velocity,Testo440 Globe Temerature
0,6/20/22 9:06,nen11,22.0,Acceptable,No change,No change,1,2,Watch 1,NaN,AC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.133654,88.05157,27.204983,718.7729,68.1084,27.249247,495.186035,71.09089,21.330507,5284.89551,568.308533,126.643692,20.0,615.6636,26.745117,226.981674,27.3,0.0,27.62047,290.635437,26.365353,679.8063,79.10984,365.392273,5.030742,3.779608,72.88983,54.851803,26.24638,20.0,645.796265,78.5718,148.35675,4.2417,3.0,73.7789,56.118713,26.202334,20.0,73.0,7.1,11.0,10.5,1010.6,83.0,27.8,73.4,0.114,734.0,0.228,1.0,0.847,27.3,74.0,0.239,689.0,0.396,1.0,1.141,27.1,26.9,0.53,27.6
1,6/20/22 9:06,nen12,26.0,Acceptable,Cooler,No change,2,2,Watch 1,NaN,AC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.133654,88.05157,27.204983,718.7729,68.1084,27.249247,495.186035,71.09089,21.330507,5284.89551,568.308533,126.643692,20.0,615.6636,26.745117,226.981674,27.3,0.0,27.62047,290.635437,26.365353,679.8063,79.10984,365.392273,5.030742,3.779608,72.88983,54.851803,26.24638,20.0,645.796265,78.5718,148.35675,4.2417,3.0,73.7789,56.118713,26.202334,20.0,73.0,7.1,11.0,10.5,1010.6,83.0,27.8,73.4,0.114,734.0,0.228,1.0,0.847,27.3,74.0,0.239,689.0,0.396,1.0,1.141,27.1,26.9,0.53,27.6
2,6/20/22 9:06,nen14,29.0,Acceptable,No change,No change,4,2,Watch 1,NaN,AC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

# Perform column relabelling

In [3]:
# rename columns to human readable format
name_mapping = {'Thermal Preference':'Thermal Preference Watch',
                'Thermal Preference.1':'Thermal Preference Online',
                'Air Movement Preference':'Air Movement Preference Watch',
                'Airflow Preference':'Air Movement Preference Online',
                'Thermal Sensation':'Thermal Sensation',
                'Thermal Acceptabil':'Thermal Acceptability Online',
                'Thermal Acceptance':'Thermal Acceptability Watch',
                'Space Preference_1':'Space Preference',
                'Air Humidity Sensati':'Air Humidity Sensation',
                'Air Humidity Prefere':'Air Humidity Preference',
                'Visual':'Visual_1',
                'Visual.1':'Visual_2',
                'BCA Test Site_AirReadings_Off Coil Temp':'Off Coil Temp',
                'BCA Test Site_AirReadings_Outdoor Air RH':'Outdoor Air RH',
                'BCA Test Site_AirReadings_Outdoor Air Temp':'Outdoor Air Temp',
                'BCA Test Site_AirReadings_Return Air CO2':'Return Air CO2',
                'BCA Test Site_AirReadings_Return Air RH':'Return Air RH',
                'BCA Test Site_AirReadings_Return Air Temp':'Return Air Temp',
                'BCA Test Site_AirReadings_Supply Air CO2':'Supply Air CO2',
                'BCA Test Site_AirReadings_Supply Air RH':'Supply Air RH',
                'BCA Test Site_AirReadings_Supply Air Temp':'Supply Air Temp',
                'BCA Test Site_BTUReadings_BTU Cooling Load W':'BTU Cooling Load',
                'BCA Test Site_MainLinePowerMeter_Power W':'MainLine Power',
                'BCA Test Site_PowerMeterReadings_AHU_Power W':'AHU Power',
                'BCA Test Site_PowerMeterReadings_Fan_Power W':'Fan Power',
                'BCA Test Site_VAVFeedback_VAV-01_Airflow Final':'VAV-01_Airflow',
                'BCA Test Site_VAVFeedback_VAV-01_Temp':'VAV-01_Temp',
                'BCA Test Site_VAVFeedback_VAV-02_Airflow Final':'VAV-02_Airflow',
                'BCA Test Site_VAVFeedback_VAV-02_Temp':'VAV-02_Temp',
                'BCA Test Site_VAVFeedback_VAV-03_Airflow Final':'VAV-03_Airflow',
                'BCA Test Site_VAVFeedback_VAV-03_Temp':'VAV-03_Temp',
                'BCA Test Site_VAVFeedback_VAV-04_Airflow Final':'VAV-04_Airflow',
                'BCA Test Site_VAVFeedback_VAV-04_Temp':'VAV-04_Temp',
                'UFL.AWAIR 33482 CO2':'Indoor CO2_1',
                'UFL.AWAIR 33482 HUMIDITY':'Indoor Humidity_1',
                'UFL.AWAIR 33482 LUX':'Indoor Lux_1',
                'UFL.AWAIR 33482 PM10':'Indoor PM10_1',
                'UFL.AWAIR 33482 PM2.5':'Indoor PM2.5_1',
                'UFL.AWAIR 33482 SCORE':'Indoor Score_1',
                'UFL.AWAIR 33482 SOUND':'Indoor Sound_1',
                'UFL.AWAIR 33482 TEMP':'Indoor Temp_1',
                'UFL.AWAIR 33482 VOC':'Indoor VOC_1',
                'UFL.AWAIR 33869 CO2':'Indoor CO2_2',
                'UFL.AWAIR 33869 HUMIDITY':'Indoor Humidity_2',
                'UFL.AWAIR 33869 LUX':'Indoor Lux_2',
                'UFL.AWAIR 33869 PM10':'Indoor PM10_2',
                'UFL.AWAIR 33869 PM2.5':'Indoor PM2.5_2',
                'UFL.AWAIR 33869 SCORE':'Indoor Score_2',
                'UFL.AWAIR 33869 SOUND':'Indoor Sound_2',
                'UFL.AWAIR 33869 TEMP':'Indoor Temp_2',
                'UFL.AWAIR 33869 VOC':'Indoor VOC_2',
                'UFL.Purple Air Outdoor 145432 Humidity':'Outdoor Humidity',
                'UFL.Purple Air Outdoor 145432 PM1':'Outdoor PM1',
                'UFL.Purple Air Outdoor 145432 PM10':'Outdoor PM10',
                'UFL.Purple Air Outdoor 145432 PM2.5':'Outdoor PM2.5',
                'UFL.Purple Air Outdoor 145432 Pressure':'Outdoor Pressure',
                'UFL.Purple Air Outdoor 145432 Temperature':'Outdoor Temp',
                'Testo400 Globe Temerature':'Globe Temperature_1',
                'Testo400 Temperature':'Temperature',  # how is this different from outdoor temp?
                'Testo400 Air Velocity':'Air Velocity',
                'Testo440 Globe Temerature':'Globe Temperature_2',
                'Demographics':'User Id',
                'Condition': 'Mode_temp'}

name_mapping_2 = {'Mode':'Condition',
                  'Mode_temp':'Mode'}

data.rename(columns=name_mapping, inplace=True)
data.rename(columns=name_mapping_2, inplace=True)
data.head()

,Timestamp,User ID,voteLog,Thermal Acceptability Watch,Thermal Preference Watch,Air Movement Preference Watch,User Id,Condition,Survey Type,StartDate,Mode,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,Thermal Sensation,Thermal Acceptability Online,Thermal Preference Online,Space Preference,Reason for Rating,Air Freshness,Airflow Sensation,Air Movement Preference Online,Air Humidity Sensation,Air Humidity Preference,Indoor Air Quality,Indoor Air Quality.1,Indoor Air Quality_10_TEXT,Acoustics_1,Acoustics_2,Acoustics_3,Visual_1,Visual_2,Off Coil Temp,Outdoor Air RH,Outdoor Air Temp,Return Air CO2,Return Air RH,Return Air Temp,Supply Air CO2,Supply Air RH,Supply Air Temp,BTU Cooling Load,MainLine Power,AHU Power,Fan Power,VAV-01_Airflow,VAV-01_Temp,VAV-02_Airflow,VAV-02_Temp,VAV-03_Airflow,VAV-03_Temp,VAV-04_Airflow,VAV-04_Temp,Indoor CO2_1,Indoor Humidity_1,Indoor Lux_1,Indoor PM10_1,Indoor PM2.5_1,Indoor Score_1,Indoor Sound_1,Indoor Temp_1,Indoor VOC_1,Indoor CO2_2,Indoor Humidity_2,Indoor Lux_2,Indoor PM10_2,Indoor PM2.5_2,Indoor Score_2,Indoor Sound_2,Indoor Temp_2,Indoor VOC_2,Outdoor Humidity,Outdoor PM1,Outdoor PM10,Outdoor PM2.5,Outdoor Pressure,Outdoor Temp,PPM55 Temperature,PPM55 Relative Humidity,PPM55 CO,PPM55 CO2,PPM55 TVOC,PPM55 Test Number,PPM55 Air Velocity,PPM54 Temperature,PPM54 Relative Humidity,PPM54 CO,PPM54 CO2,PPM54 TVOC,PPM54 Test Number,PPM54 Air Velocity,Globe Temperature_1,Temperature,Air Velocity,Globe Temperature_2
0,6/20/22 9:06,nen11,22.0,Acceptable,No change,No change,1,2,Watch 1,NaN,AC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.133654,88.05157,27.204983,718.7729,68.1084,27.249247,495.186035,71.09089,21.330507,5284.89551,568.308533,126.643692,20.0,615.6636,26.745117,226.981674,27.3,0.0,27.62047,290.635437,26.365353,679.8063,79.10984,365.392273,5.030742,3.779608,72.88983,54.851803,26.24638,20.0,645.796265,78.5718,148.35675,4.2417,3.0,73.7789,56.118713,26.202334,20.0,73.0,7.1,11.0,10.5,1010.6,83.0,27.8,73.4,0.114,734.0,0.228,1.0,0.847,27.3,74.0,0.239,689.0,0.396,1.0,1.141,27.1,26.9,0.53,27.6
1,6/20/22 9:06,nen12,26.0,Acceptable,Cooler,No change,2,2,Watch 1,NaN,AC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.133654,88.05157,27.204983,718.7729,68.1084,27.249247,495.186035,71.09089,21.330507,5284.89551,568.308533,126.643692,20.0,615.6636,26.745117,226.981674,27.3,0.0,27.62047,290.635437,26.365353,679.8063,79.10984,365.392273,5.030742,3.779608,72.88983,54.851803,26.24638,20.0,645.796265,78.5718,148.35675,4.2417,3.0,73.7789,56.118713,26.202334,20.0,73.0,7.1,11.0,10.5,1010.6,83.0,27.8,73.4,0.114,734.0,0.228,1.0,0.847,27.3,74.0,0.239,689.0,0.396,1.0,1.141,27.1,26.9,0.53,27.6
2,6/20/22 9:06,nen14,29.0,Acceptable,No change,No change,4,2,Watch 1,NaN,AC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.133654,88.05157,27.204983,718.7729,68.1084,27.249247,495.186035,71.09089,21.330507,5284.89551,568.308533,126.643692,20.0,615.6636,26.745117,226.981674,27.3,0.0,27.62047,290.635437,26.365353,679.8063,79.10984,365.392273,5.030742,3.779608,72.88983,54.851803,26.24638,20.0,645.796265,78.5718,148.35675,4.2417,3.0,73.7789,56.118713,26.202334,20.0,73.0,7.1,11.0,10.5,1010.6,83.0,27.8,73.4,0.114,734.0,0.228,1.0,0.847,27.3,74.0,0.239,689.0,0.396,1.0,1.141,27.1,26.9,0.53,27.6
3,6/20/22 9:06,nen15,17.0,Unacceptable,Cooler,More,5,2,Watch 1,NaN,AC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.133654,88.05157,27.204983,718.7729,68.1084,27.249247,495.186035,71.09089,21.330507,5284.89551,568.308533,126.643692,2

# Perform data cleaning

In [4]:
# combine readings from multiple sensors within the study area
def combine_sensor_readings(data, sensor_1, sensor_2, new_col_name):
    data[new_col_name] = (data[sensor_1] + data[sensor_2]) / 2
    data.drop(columns=[sensor_1, sensor_2], inplace=True)
    return data

data = combine_sensor_readings(data, 'Indoor CO2_1', 'Indoor CO2_2', 'Indoor CO2')
data = combine_sensor_readings(data, 'Indoor Humidity_1', 'Indoor Humidity_2', 'Indoor Humidity')
data = combine_sensor_readings(data, 'Indoor Lux_1', 'Indoor Lux_2', 'Indoor Lux')
data = combine_sensor_readings(data, 'Indoor PM10_1', 'Indoor PM10_2', 'Indoor PM10')
data = combine_sensor_readings(data, 'Indoor PM2.5_1', 'Indoor PM2.5_2', 'Indoor PM2.5')
data = combine_sensor_readings(data, 'Indoor Score_1', 'Indoor Score_2', 'Indoor Score')
data = combine_sensor_readings(data, 'Indoor Sound_1', 'Indoor Sound_2', 'Indoor Sound')
data = combine_sensor_readings(data, 'Indoor Temp_1', 'Indoor Temp_2', 'Indoor Temp')
data = combine_sensor_readings(data, 'Indoor VOC_1', 'Indoor VOC_2', 'Indoor VOC')
data = combine_sensor_readings(data, 'Globe Temperature_1', 'Globe Temperature_2', 'Globe Temperature')

In [5]:
# Combine air movement and thermal preference preferences
data['Thermal Preference'] = data['Thermal Preference Watch'].fillna(data['Thermal Preference Online'])
data['Air Movement Preference'] = data['Air Movement Preference Watch'].fillna(data['Air Movement Preference Online'])
data['Thermal Acceptability'] = data['Thermal Acceptability Watch'].fillna(data['Thermal Acceptability Online'])


In [6]:
# Standardise preference labels
data["Thermal Preference"] = data["Thermal Preference"].replace("No change", "No Change")
data['Air Movement Preference'] = data["Air Movement Preference"].replace("No change", "No Change")

data['Thermal Acceptability'] = data["Thermal Acceptability"].replace("Clearly Acceptable", "Acceptable")
data['Thermal Acceptability'] = data["Thermal Acceptability"].replace("Just Acceptable", "Acceptable")
data['Thermal Acceptability'] = data["Thermal Acceptability"].replace("Clearly Unacceptable", "Unacceptable")
data['Thermal Acceptability'] = data["Thermal Acceptability"].replace("Just Unacceptable", "Unacceptable")

# Filter out relevant data to ASHRAE dataset

In [7]:
# Retain overlapping columns with ASHRAE dataset
overlapping_cols = ['Timestamp',
                    'Mode',
                    'Indoor Temp',
                    'Indoor Humidity',
                    'Air Velocity',
                    'Globe Temperature',
                    'Outdoor Temp',
                    'Outdoor Humidity',
                    'Thermal Preference',
                    'Thermal Acceptability',
                    'Air Movement Preference',
                    'User Id',
                    'Condition']

data = data[overlapping_cols]

In [8]:
# Filter conditions related to AC
data = data[data["Mode"].isin(["AC", "AC+PV", "NV"])].reset_index(drop=True)
data['Mode'] = data['Mode'].replace('AC+PV', 'AC')

In [9]:
# remove condition 0
data = data[data['Condition']>0].reset_index(drop=True)

In [10]:
# map conditions from AC + PV into AC (5->2, 6->3, 7->4)
data['Condition'] = data['Condition'].replace(5, 2)
data['Condition'] = data['Condition'].replace(6, 3)
data['Condition'] = data['Condition'].replace(7, 4)

In [11]:
data['Condition'].unique()

array([ 2,  1,  4, 16, 17, 19, 18,  3])

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1744 entries, 0 to 1743
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Timestamp                1744 non-null   object 
 1   Mode                     1744 non-null   object 
 2   Indoor Temp              1742 non-null   float64
 3   Indoor Humidity          1742 non-null   float64
 4   Air Velocity             1742 non-null   float64
 5   Globe Temperature        1742 non-null   float64
 6   Outdoor Temp             1742 non-null   float64
 7   Outdoor Humidity         1742 non-null   float64
 8   Thermal Preference       1744 non-null   object 
 9   Thermal Acceptability    1744 non-null   object 
 10  Air Movement Preference  1744 non-null   object 
 11  User Id                  1744 non-null   int64  
 12  Condition                1744 non-null   int64  
dtypes: float64(6), int64(2), object(5)
memory usage: 177.2+ KB


# Perform imputation for missing data using MissForest

In [13]:
def impute_missing_data(data):    
    original_col_sequence = list(data.columns)
    imputer = MissForest(criterion='squared_error')
    numerical_data = data.drop(columns=['Timestamp',
                                        'Mode',
                                        'Thermal Preference',
                                        'Thermal Acceptability',
                                        'Air Movement Preference'])
    numerical_data_cols = list(numerical_data.columns)
    imputed_numerical_data = imputer.fit_transform(numerical_data)
    imputed_numerical_data = pd.DataFrame(imputed_numerical_data, 
                                          columns=numerical_data_cols)
    imputed_data = pd.concat([data[['Timestamp',
                                    'Mode',
                                    'Thermal Preference',
                                    'Thermal Acceptability',
                                    'Air Movement Preference']], 
                              imputed_numerical_data], 
                             axis=1)
    imputed_data = imputed_data[original_col_sequence]

    assert imputed_data.shape == data.shape
    assert imputed_data["Timestamp"].tolist() == data["Timestamp"].tolist()
    assert imputed_data.isnull().values.sum() == 0

    return imputed_data

data = impute_missing_data(data)

Iteration: 0
Iteration: 1
Iteration: 2


In [14]:
# sort entries based on timestamp
data['DateTime'] = pd.to_datetime(data['Timestamp'], format='%m/%d/%y %H:%M')
data = data.sort_values(by='DateTime').reset_index(drop=True)
data.drop(columns=['DateTime','Timestamp'], inplace=True)

In [15]:
print(data['Thermal Preference'].unique())
print(data['Air Movement Preference'].unique())
print(data['Thermal Acceptability'].unique())
print(data['Mode'].unique())
print(data.shape)
data.head()

['No Change' 'Cooler' 'Warmer']
['No Change' 'More' 'Less']
['Acceptable' 'Unacceptable']
['AC' 'NV']
(1744, 12)


,Mode,Indoor Temp,Indoor Humidity,Air Velocity,Globe Temperature,Outdoor Temp,Outdoor Humidity,Thermal Preference,Thermal Acceptability,Air Movement Preference,User Id,Condition
0,AC,26.224357,78.84082,0.53,27.35,83.0,73.0,No Change,Acceptable,No Change,1.0,2.0
1,AC,26.224357,78.84082,0.53,27.35,83.0,73.0,Cooler,Acceptable,No Change,2.0,2.0
2,AC,26.224357,78.84082,0.53,27.35,83.0,73.0,No Change,Acceptable,No Change,4.0,2.0
3,AC,26.224357,78.84082,0.53,27.35,83.0,73.0,Cooler,Unacceptable,More,5.0,2.0
4,AC,26.224357,78.84082,0.53,27.35,83.0,73.0,Cooler,Acceptable,More,6.0,2.0


# Save processed data

In [16]:
# thermal preference data
thermal_pref_data = data.drop(columns=['Thermal Acceptability','Air Movement Preference'])
thermal_pref_data = thermal_pref_data.dropna(subset=['Thermal Preference']).reset_index(drop=True)
assert thermal_pref_data.isnull().values.any() == False
thermal_pref_data.to_csv('data/bca_thermalpref_unsampled_data.csv', index=False)

In [17]:
# thermal acceptability data
thermal_acc_data = data.drop(columns=['Thermal Preference','Air Movement Preference'])
thermal_acc_data = thermal_acc_data.dropna(subset=['Thermal Acceptability']).reset_index(drop=True)
assert thermal_acc_data.isnull().values.any() == False
thermal_acc_data.to_csv('data/bca_thermalacc_unsampled_data.csv', index=False)

In [18]:
# air movement preference data
air_pref_data = data.drop(columns=['Thermal Acceptability','Thermal Preference'])
air_pref_data = air_pref_data.dropna(subset=['Air Movement Preference']).reset_index(drop=True)
assert air_pref_data.isnull().values.any() == False
air_pref_data.to_csv('data/bca_airpref_unsampled_data.csv', index=False)